<a href="https://colab.research.google.com/github/kmkarakaya/ML_tutorials/blob/master/LSTM_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
#@title Version Info
print('tf version: ', tf.__version__)
print('tf.keras version:', tf.keras.__version__)


tf version:  2.2.0
tf.keras version: 2.3.0-tf


#References:
[tf.keras.layers.LSTM official website](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)

[The base example](https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/)

[My Presentation in PPT format]()


In [ ]:
#@title Import Libraries
from random import randint
from numpy import array
from numpy import argmax

from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector

In [ ]:
#@title Generate one_hot_encoded Input & Output Sequences


# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(0, n_unique-1) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# prepare data for the LSTM
def get_pair(n_in, n_out, n_unique, verbose= False):
	# generate random sequence
	sequence_in = generate_sequence(n_in, n_unique)
	sequence_out = sequence_in[:n_out] + [0 for _ in range(n_in-n_out)]
	# one hot encode
	X = one_hot_encode(sequence_in, n_unique)
	y = one_hot_encode(sequence_out, n_unique)
	# reshape as 3D
	X = X.reshape((1, X.shape[0], X.shape[1]))
	y = y.reshape((1, y.shape[0], y.shape[1]))

	if(verbose):
		print('Generated sequences as follows')
		print('X.shape: ', X.shape,'y.shape: ', y.shape)
		print('\nSample X and y')
		print('\nIn raw format:')
		print('X=%s, y=%s' % (one_hot_decode(X[0]), one_hot_decode(y[0])))
		print('\nIn one_hot_encoded format:')
		print('X=%s' % (X[0]))
	return X,y

In [ ]:
#@title Configure problem

n_timesteps_in = 4  #@param {type:"integer"}
#each input sample has 4 values

n_features = 10   #@param {type:"integer"}
#each value is one_hot_encoded with 10 0/1
n_timesteps_out = 2  #@param {type:"integer"}
#each output sample has 2 values padded with 0

# generate random sequence
X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features, verbose=True)

Generated sequences as follows
X.shape:  (1, 4, 10) y.shape:  (1, 4, 10)

Sample X and y

In raw format:
X=[6, 2, 3, 7], y=[6, 2, 0, 0]

In one_hot_encoded format:
X=[[0 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]]


In [ ]:
# define model
numberOfLSTMcells= 100

input =Input(shape=(n_timesteps_in, n_features))
output= LSTM(numberOfLSTMcells, return_sequences=True) (input)
model1 = Model(inputs=input, outputs=output)
model1.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 4, 10)]           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 4, 100)            44400     
Total params: 44,400
Trainable params: 44,400
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#lstm, state_h, state_c

In [ ]:
# define model
model = Sequential()
model.add(Input(shape=(n_timesteps_in, n_features)))
model.add(LSTM(150))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(150, return_sequences=True))
model.add(TimeDistributed(Dense(n_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 150)               96600     
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 4, 150)            0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 4, 150)            180600    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4, 10)             1510      
Total params: 278,710
Trainable params: 278,710
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train LSTM
for epoch in range(500):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, verbose=2)
	print(epoch,' finished...')

In [ ]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Accuracy: 28.00%


In [ ]:
# spot check some examples
for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	print('Input',one_hot_decode(X[0]),
	      'Expected:', one_hot_decode(y[0]), 
				   'Predicted', one_hot_decode(yhat[0]))

Input [7, 6, 2, 5] Expected: [7, 6, 0, 0] Predicted [7, 7, 0, 0]
Input [5, 8, 1, 9] Expected: [5, 8, 0, 0] Predicted [5, 5, 0, 0]
Input [4, 8, 4, 1] Expected: [4, 8, 0, 0] Predicted [4, 4, 0, 0]
Input [7, 2, 7, 1] Expected: [7, 2, 0, 0] Predicted [7, 2, 0, 0]
Input [7, 7, 8, 8] Expected: [7, 7, 0, 0] Predicted [7, 7, 0, 0]
Input [4, 6, 3, 4] Expected: [4, 6, 0, 0] Predicted [4, 4, 0, 0]
Input [1, 1, 3, 2] Expected: [1, 1, 0, 0] Predicted [1, 1, 0, 0]
Input [8, 4, 5, 7] Expected: [8, 4, 0, 0] Predicted [8, 8, 0, 0]
Input [0, 2, 3, 3] Expected: [0, 2, 0, 0] Predicted [3, 0, 0, 0]
Input [0, 0, 5, 2] Expected: [0, 0, 0, 0] Predicted [0, 0, 0, 0]


In [ ]:
x= tf.constant([[3,4,5]])
print(x)
tf.keras.layers.RepeatVector(3)(x)

In [ ]:
# define model
model3 = Sequential()
model3.add(Input(shape=(n_timesteps_in, n_features)))
model3.add(LSTM(150, return_sequences=True))
model3.add(LSTM(150, return_sequences=True))
model3.add(TimeDistributed(Dense(n_features, activation='softmax')))
model3.compile(loss='categorical_crossentropy', optimizer='adam', 
               metrics=['accuracy'])
model3.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 4, 150)            96600     
_________________________________________________________________
lstm_19 (LSTM)               (None, 4, 150)            180600    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 4, 10)             1510      
Total params: 278,710
Trainable params: 278,710
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train LSTM
for epoch in range(1500):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model3.fit(X, y, epochs=1, verbose=2)
	print(epoch,' finished...')

In [ ]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model3.predict(X, verbose=0)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Accuracy: 100.00%


In [ ]:
# spot check some examples
for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model3.predict(X, verbose=0)
	print('Input',one_hot_decode(X[0]),
	      'Expected:', one_hot_decode(y[0]), 
				   'Predicted', one_hot_decode(yhat[0]))

Input [3, 2, 7, 6] Expected: [3, 2, 0, 0] Predicted [3, 2, 0, 0]
Input [1, 2, 7, 0] Expected: [1, 2, 0, 0] Predicted [1, 2, 0, 0]
Input [2, 7, 3, 9] Expected: [2, 7, 0, 0] Predicted [2, 7, 0, 0]
Input [8, 1, 4, 4] Expected: [8, 1, 0, 0] Predicted [8, 1, 0, 0]
Input [5, 6, 4, 2] Expected: [5, 6, 0, 0] Predicted [5, 6, 0, 0]
Input [6, 1, 8, 8] Expected: [6, 1, 0, 0] Predicted [6, 1, 0, 0]
Input [5, 3, 5, 4] Expected: [5, 3, 0, 0] Predicted [5, 3, 0, 0]
Input [6, 3, 4, 6] Expected: [6, 3, 0, 0] Predicted [6, 3, 0, 0]
Input [9, 1, 5, 0] Expected: [9, 1, 0, 0] Predicted [9, 1, 0, 0]
Input [6, 0, 3, 5] Expected: [6, 0, 0, 0] Predicted [6, 0, 0, 0]
